## **목표**
딥러닝의 중요 개념 중 하나인 **전이학습(Transfer Learning)**은 이미 학습된 모델의 지식을 새로운 학습에 활용하는 방법으로, 효율적이고 빠른 학습을 가능하게 한다.    
PyTorch에서는 다양한 **사전 학습된 모델**을 제공하고 있으며, 이런 모델들을 기반으로 전이학습을 수행할 수 있다.   
PyTorch에서 제공되는 모델들을 저장하고 불러와 보고, 데이터셋을 통해 전이 학습을 실습해본다.

In [ ]:
!pip install torchsummary

**❓퀴즈**   
### **torch.save()**
- 학습의 결과를 저장하기 위한 함수
- **🖊 정답:** 모델 형태(architecture)와 파라미터(가중치)를 저장
- 중간 과정의 저장을 통해 최선의 결과 모델을 선택, 만든 모델을 외부 연구자와 공유하여 삭습 재연성 향상

### **checkpoints**
- 학습의 중간 결과를 저장하여 최선의 결과를 선택
- Early Stopping 기법 사용시 이전 학습의 결과물을 저장
- loss, metric값을 지속적으로 확인, 저장
    - 일반적으로 epoch, loss, metric을 함께 저장하여 확인

### **1. PyTorch 모델 저장하기 & 불러오기**

In [ ]:
import os
import shutil
import zipfile
import warnings
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchsummary import summary
from os import walk
from PIL import Image
from torchvision import datasets
from torchvision import models

warnings.filterwarnings("ignore")

In [ ]:
# 신경망 모델 클래스 정의
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()

        # 첫 번째 층: 합성곱, 배치 정규화, ReLU 활성화 함수, 최대 풀링 포함
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        # 두 번째 층
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        # 세 번째 층
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        # 드롭아웃 층
        self.drop_out = nn.Dropout()

        # 완전 연결 층
        self.fc1 = nn.Linear(3 * 3 * 64, 1000)
        self.fc2 = nn.Linear(1000, 1)

    # 순전파 함수 정의
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)

        # 텐서를 1차원으로 변환
        out = out.view(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
# 모델 초기화 및 GPU로 이동.
model = TheModelClass().cuda()

# optimizer 초기화, SGD 사용
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

In [ ]:
# 모델 상태 사전 출력
# ordered dict 타입으로 출력됨.
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
layer1.0.weight 	 torch.Size([16, 3, 3, 3])
layer1.0.bias 	 torch.Size([16])
layer1.1.weight 	 torch.Size([16])
layer1.1.bias 	 torch.Size([16])
layer1.1.running_mean 	 torch.Size([16])
layer1.1.running_var 	 torch.Size([16])
layer1.1.num_batches_tracked 	 torch.Size([])
layer2.0.weight 	 torch.Size([32, 16, 3, 3])
layer2.0.bias 	 torch.Size([32])
layer2.1.weight 	 torch.Size([32])
layer2.1.bias 	 torch.Size([32])
layer2.1.running_mean 	 torch.Size([32])
layer2.1.running_var 	 torch.Size([32])
layer2.1.num_batches_tracked 	 torch.Size([])
layer3.0.weight 	 torch.Size([64, 32, 3, 3])
layer3.0.bias 	 torch.Size([64])
layer3.1.weight 	 torch.Size([64])
layer3.1.bias 	 torch.Size([64])
layer3.1.running_mean 	 torch.Size([64])
layer3.1.running_var 	 torch.Size([64])
layer3.1.num_batches_tracked 	 torch.Size([])
fc1.weight 	 torch.Size([1000, 576])
fc1.bias 	 torch.Size([1000])
fc2.weight 	 torch.Size([1, 1000])
fc2.bias 	 torch.Size([1])


In [ ]:
# 모델 상태 사전 타입 출력
type(model.state_dict())

collections.OrderedDict

In [ ]:
# 모델의 구조 및 파라미터 요약 출력
# summary로 상태 사전을 케라스 형태로 확인할 수 있음.
# 이렇게 보는 것을 권장
summary(model, (3, 244, 244))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 121, 121]             448
       BatchNorm2d-2         [-1, 16, 121, 121]              32
              ReLU-3         [-1, 16, 121, 121]               0
         MaxPool2d-4           [-1, 16, 60, 60]               0
            Conv2d-5           [-1, 32, 29, 29]           4,640
       BatchNorm2d-6           [-1, 32, 29, 29]              64
              ReLU-7           [-1, 32, 29, 29]               0
         MaxPool2d-8           [-1, 32, 14, 14]               0
            Conv2d-9             [-1, 64, 6, 6]          18,496
      BatchNorm2d-10             [-1, 64, 6, 6]             128
             ReLU-11             [-1, 64, 6, 6]               0
        MaxPool2d-12             [-1, 64, 3, 3]               0
          Dropout-13                  [-1, 576]               0
           Linear-14                 [-

In [ ]:
# 모델 저장 경로
MODEL_PATH = "saved"

# 지정된 경로가 존재하지 않으면, 해당 경로를 생성
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

# 모델의 상태 사전을 저장.
torch.save(model.state_dict(), os.path.join(MODEL_PATH, 'model.pt'))

In [ ]:
# 모델 불러오기
# 동일한 모델이어야 함
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

new_model = TheModelClass()
new_model.load_state_dict(torch.load(os.path.join(MODEL_PATH, 'model.pt')))

<All keys matched successfully>

In [ ]:
# 전체 모델을 파일로 저장(pickle)
torch.save(model, os.path.join(MODEL_PATH, 'model_pickle.pt'))

# 저장된 전체 모델 불러오기
model = torch.load(os.path.join(MODEL_PATH, 'model_pickle.pt'))

# 모델을 평가 모드로 설정. (드롭 아웃 및 배치 정규화 비활성화)
model.eval()

TheModelClass(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (drop_out): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=576, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=1, b

In [ ]:
# 개와 고양이 데이터셋 가져오기
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

In [ ]:
# 드라이브 마운트 해서 가져오는게 더 빠름.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 압축 파일명 지정
filename = 'kagglecatsanddogs_5340.zip'

# 압축 파일 풀기
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall()

# 'PetImages' 폴더를 'data'폴더로 이동
shutil.move('PetImages', 'data')

# 'data' 폴더 내의 모든 파일 경로를 가져오기
my_path='data'
f_path=[]
for(dirpath, dirnames, filenames) in walk(my_path):
    f_path.extend(os.path.join(dirpath, filename) for filename in filenames)

# 손상된 이미지 파일이 있는지 확인, 손상된 파일은 삭제.
for f_name in f_path:
    try:
        Image.open(f_name)
    except Exception as e:
        print(e)
        os.remove(f_name)

cannot identify image file 'data/Dog/11702.jpg'
cannot identify image file 'data/Dog/Thumbs.db'
cannot identify image file 'data/Cat/666.jpg'
cannot identify image file 'data/Cat/Thumbs.db'


In [ ]:
# 데이터셋 로드, 전처리
dataset = datasets.ImageFolder(
                                root='data',
                                transform=transforms.Compose([
                                    transforms.Resize(244),
                                    transforms.CenterCrop(244),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5,0.5,0.5),
                                                         (0.5,0.5,0.5))
                                ]))
dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size=8,
                                         shuffle=True,
                                         num_workers=8)

In [ ]:
# 이진 분류 정확도를 계산하는 함수
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    return acc

In [ ]:
EPOCHS = 10
BATCH_SIZE = 64
LEARNING_RATE = 0.1

In [ ]:
model.to(device)

# nn.BCEWithLogitsLoss:
# 맨 끝 층에 sigmoid가 없는 모델의 경우,
# sigmoid가 없어도 자동으로 나온 값에 sigmoid를 적용시켜주는 Loss함수.
# binary classification에 이용
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).type(torch.cuda.FloatTensor)

        optimizer.zero_grad()
        y_pred = model(X_batch)
        # y_pred=(batch_size, 1), y_batch=(batch_size)
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    # 모델의 상태와 옵티마이저의 상태를 저장.
    # 이렇게 saved 폴더에 저장하는 것 보다는, 구글 드라이브 안의 폴더에 저장하는 것이 좋다.
    # loss가 좋아지면 저장한다거나, acc가 좋아지면 저장한다거나,
    # 더 이상 loss가 줄지 않고 증가하면 멈춘다거나 이런 식으로 활용할 수 있다.
    # 모델 저장 시 .pth보다는 .pt를 쓰는 것을 추천.
    torch.save(
        {
            'epoch':e,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': epoch_loss
        },f"saved/checkpoint_model_{e}_{epoch_loss/len(dataloader)}_{epoch_acc/len(dataloader)}.pt")

    # 에폭별 손실과 정확도를 출력.
    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(dataloader):.5f} | Acc: {epoch_acc/len(dataloader):.3f}')

Epoch 001: | Loss: 0.84945 | Acc: 49.935
Epoch 002: | Loss: 0.87515 | Acc: 49.725
Epoch 003: | Loss: 0.87216 | Acc: 50.094
Epoch 004: | Loss: 0.81532 | Acc: 49.968
Epoch 005: | Loss: 0.85331 | Acc: 49.544
Epoch 006: | Loss: 0.84168 | Acc: 50.053
Epoch 007: | Loss: 0.84857 | Acc: 50.092
Epoch 008: | Loss: 0.85844 | Acc: 50.081
Epoch 009: | Loss: 0.84216 | Acc: 50.503
Epoch 010: | Loss: 0.84573 | Acc: 49.835


### **2. Pretrained model 불러오기**


### **Transfer learning**
이미 학습된 모델을 학습하고자 하는 다른 데이터에 맞춰 학습시키는 것. 대용량 데이터셋으로 만들어진 모델로, 현재 데이터셋(대용량 데이터셋의 일부)를 학습시킨다면 높은 정확도가 나올 것임.     

현재 DL에서 가장 일반적인 학습 방법.   

Backbone architecture가 잘 학습된 모델에서 일부분만 변경하여 학습을 수행.   

**TorchVision**에서는 다양한 기본 모델을 제공하고 있음.
<br><br>

### **Freezing**
- Pretrained model 활용시 모델의 일부분을 frozen시킴.
    - 더 이상 모델 파라미터 값을 바뀌지 않게 하는 것.
    - backpropagation이 나머지 일부분에서만 일어나게 함.


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 사전 학습된 VGG16 모델 불러오기
vgg = models.vgg16(pretrained=True).to(device)
print(vgg)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:09<00:00, 60.3MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
summary(vgg, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [ ]:
# 방법 1. 대부분 이 방법을 사용
# VGG16 모델의 각 층의 이름과 구조를 출력
for name, layer in vgg.named_modules():
    print(name, layer)

 VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=

In [ ]:
# VGG 모델의 완전 연결 층(FC)을 수정해 출력 유닛 수를 1로 변경.
vgg.fc = torch.nn.Linear(1000,1)
vgg.cuda()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# 방법 2,
# 다시 사전 학습된 VGG16 모델 불러오기
vgg = models.vgg16(pretrained=True).to(device)
print(vgg)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# VGG16모델의 분류기(Classifier) 중 마지막 선형 층을 수정하여 출력 유닛의 수를 1로 변경.
vgg.classifier._modules['6'] = torch.nn.Linear(4096, 1)
vgg.cuda()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### **3. Pretrained Model로 나만의 모델 만들기**

In [ ]:
# 'data/' 경로의 이미지 데이터를 불러오고 전처리를 적용.
dataset = datasets.ImageFolder(
                                root = 'data/',
                                transform = transforms.Compose([
                                    transforms.Resize(224),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5),
                                                         (0.5, 0.5, 0.5))
                                ]))
#num_workers는 dataset의 데이터를 gpu로 전송할 때 필요한 전처리를 수행할 때 사용하는 subprocess의 수를 말한다.
# num_workers의 수를 늘리면 병렬처리를 통해 더 빠르게 gpu에 정보를 전달할 수 있어 성능이 좋아진다.
# 하지만 num_workers의 수가 너무 크면 다른 일을 수행하는 데 사용할 자원이 적어져 성능이 안좋아질 수 있다.
# 따라서 적절한 값을 찾기 위해 하이퍼파라미터 튜닝을 하듯 접근한다.
dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, num_workers=8)

In [ ]:
# 새로운 신경망 모델 정의
class MyNewNet(nn.Module):
    def __init__(self):
        super(MyNewNet, self).__init__()
        # 사전 학습된 VGG19 사용.
        self.vgg19 = models.vgg19(pretrained=True)
        # 추가적인 선형 층 정의
        self.linear_layers = nn.Linear(1000, 1)

    def forward(self, x):
        x = self.vgg19(x)
        return self.linear_layers(x)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화, GPU로 이동
my_model = MyNewNet()
my_model = my_model.to(device)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:05<00:00, 103MB/s]


In [ ]:
print(my_model)

MyNewNet(
  (vgg19): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16):

In [ ]:
# 모델의 모든 파라미터 고정(frozen)
for param in my_model.parameters():
    param.requires_grad = False

In [ ]:
# 선형 층의 파라미터만 학습 가능하도록 설정
for param in my_model.linear_layers.parameters():
    param.requires_grad = True

In [ ]:
# 모델을 평가 모드로 설정
my_model.eval()

MyNewNet(
  (vgg19): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16):

In [ ]:
# 데이터 로더에서 첫 번째 배치 가져오기
x = next(iter(dataloader))

In [ ]:
# 입력 데이터의 형태를 출력.
x[0].shape

torch.Size([2, 3, 224, 224])

In [ ]:
# 모델을 통해 예측 수행
my_model(x[0].to(device))

tensor([[1.1347],
        [0.7828]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
EPOCHS = 5
BATCH_SIZE = 64
LEARNING_RATE = 0.001

In [ ]:
# 데이터 로더의 배치 크기를 업데이트.
dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size=BATCH_SIZE,
                                         shuffle=True,
                                         num_workers=8)

In [ ]:
# VGG19 모델의 첫 번재 합성곱 층의 파라미터를 출력.
next(my_model.vgg19.features._modules['0'].parameters())[0]

tensor([[[-0.0535, -0.0493, -0.0679],
         [ 0.0153,  0.0451,  0.0021],
         [ 0.0362,  0.0200,  0.0199]],

        [[ 0.0170,  0.0554, -0.0062],
         [ 0.1416,  0.2271,  0.1376],
         [ 0.1200,  0.2003,  0.0921]],

        [[-0.0449,  0.0127, -0.0145],
         [ 0.0597,  0.1395,  0.0541],
         [-0.0010,  0.0583, -0.0297]]], device='cuda:0')

In [ ]:
# 선형 층의 파라미터 중 첫 10개를 출력
it = my_model.linear_layers.parameters()
next(it)[0][:10]

tensor([-0.0254,  0.0216, -0.0235, -0.0238, -0.0080,  0.0051, -0.0244, -0.0021,
         0.0127, -0.0067], device='cuda:0', grad_fn=<SliceBackward0>)

In [ ]:
# vgg19를 쓰기 때문에 학습 속도가 느리다..
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    return acc

# 모델 선언도 다시 해 주는게 좋음.
my_model = MyNewNet()
my_model = my_model.to(device)

# 모델의 모든 파라미터를 고정.
for param in my_model.parameters():
    param.requires_grad = False

# 선형 층의 파라미터만 학습 가능하도록 설정.
for param in my_model.linear_layers.parameters():
    param.requires_grad = True

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(my_model.parameters(), lr=LEARNING_RATE)

for e in range(1, EPOCHS+1):
    epoch_loss=0
    epoch_acc=0

    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).type(torch.cuda.FloatTensor)

        optimizer.zero_grad()
        y_pred = my_model(X_batch)

        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() # torch.Tensor.item(). 단일 값의 텐서에서 Python 숫자를 추출하는 데 사용.
                                  # 해당 메서드는 텐서가 1개의 요소만 가질 때 호출 가능.
        epoch_acc += acc.item()


    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(dataloader):.5f} | Acc: {epoch_acc/len(dataloader):.3f}')

Epoch 001: | Loss: 0.06760 | Acc: 97.240
Epoch 002: | Loss: 0.05129 | Acc: 97.898
Epoch 003: | Loss: 0.05116 | Acc: 98.013
Epoch 004: | Loss: 0.05055 | Acc: 98.051
Epoch 005: | Loss: 0.05139 | Acc: 98.028


In [ ]:
# VGG19 모델의 첫 번째 합성곱 층의 파라미터 출력(frozen)
next(my_model.vgg19.features._modules['0'].parameters())[0] # 첫 번째 파라미터를 가져옴

tensor([[[-0.0535, -0.0493, -0.0679],
         [ 0.0153,  0.0451,  0.0021],
         [ 0.0362,  0.0200,  0.0199]],

        [[ 0.0170,  0.0554, -0.0062],
         [ 0.1416,  0.2271,  0.1376],
         [ 0.1200,  0.2003,  0.0921]],

        [[-0.0449,  0.0127, -0.0145],
         [ 0.0597,  0.1395,  0.0541],
         [-0.0010,  0.0583, -0.0297]]], device='cuda:0')

In [ ]:
# 선형 층의 파라미터 중 첫 10개를 출력(frozen 시키지 않음)
it = my_model.linear_layers.parameters()
next(it)[0][:10]

tensor([-0.0341, -0.0572,  0.0226, -0.0207, -0.0028, -0.0196,  0.0055, -0.0002,
         0.0161, -0.0171], device='cuda:0', grad_fn=<SliceBackward0>)